# Starting the Game Model


## Combinação de Modelos
- Modelo 1 - Random Forest Classifier (ALL)

Se não encontrar WIN, vai pro próximo
- Modelo 2 - Random Forest Classifier (col by col)

Se Não encontrar WIN, vai pro próximo
- Model 3 - Rede Neural

Se Não encontrar, vamos apelar para uma jogada aleatória

- Modelo 4 - Aleatório (Aqui excluímos as piores posições [2,4,6,8] ) e fazemos o soteio


### Lógica a busca pela vitória
exemplo:

Turno 1 - Meu turno

(1_,'_','_','_','_','_','_','_','_')  Joguei

Turno 2 - Máquina

Ela irá jogar o próximo valor em todas posições possíveis não ocupadas, exemplo

(1_,'2','_','_','_','_','_','_','_') = 0 

(1_,'_','2','_','_','_','_','_','_') = 0 
.
.
.
(1_,'_','_','_','_','_','_','_','2') = 0 

Então irá examinar... se ocorrer vitória (1) ou empate (0.5), ela joga nesta posição o 2, caso contrário, ela testa outro número


Caso ele não encontre 1 ou 0.5... ele joga um valor aleatório removendo os valores
[2,4,6,8] dos valores LIVRES... pois são localizações ruins

- ~Toda vez que ele check se há vitória, derrota ou empate... caso dê okay, ele faz o treinamento da rede neural com novos valores inputados!~ (fazia na rede neural


## OBS: Lances indefinidos quando resultam em Win

é perigoso, pois o modelo não tem muitos dados para analisar ainda... logo ele faz um chute, então que seja um chute bom... por isso, utilize a função "sortear_pos_boa()", pois pelo menos depois da segunda rodada, a máquina pode matar o jogo

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from sklearn.ensemble import RandomForestClassifier

import random
import pickle

### Random Forest Classifier MODELS

##### Load <span style="color:red">Model</span>

In [2]:
# Random Forest Classifier All
model_all = pickle.load(open('../02_train/save_model/random_forest_classifier_all.sav','rb'))

In [3]:
test = [
    [1,3,5,0,0,0.,2,4,0], # Win
    [1,3,0,5,0,0,2,4,6], # Lost
    [6,1,2,9,3,4,5,8,7], # Draw
    [2,4,0,0,0,0,1,3,5], # Win
    [1,3,0,0,5,0,2,4,6], # Lost
    [1,3,0,5,0,6,2,4,0], # Vou vencer na pŕoxima
    [1,0,0,0,0,0,0,0,0], # Indefinido (Perigoso dar Vitória.. isso não é bom)
    [2,0,3,0,0,0,0,1,0], # Indefinido (Perigoso dar Vitória.. isso não é bom)
]
test = np.array(test)
test.shape
print(model_all.predict(test))

['win' 'lost' 'draw' 'win' 'lost' 'lost' 'win' 'win']


##### Load <span style="color:red">Model</span>

In [4]:
# Random Forest Classifier (por Coluna)
model123 = pickle.load(open('../02_train/save_model/random_forest_classifier_123.sav','rb'))
model456 = pickle.load(open('../02_train/save_model/random_forest_classifier_456.sav','rb'))
model789 = pickle.load(open('../02_train/save_model/random_forest_classifier_789.sav','rb'))
model147 = pickle.load(open('../02_train/save_model/random_forest_classifier_147.sav','rb'))
model258 = pickle.load(open('../02_train/save_model/random_forest_classifier_258.sav','rb'))
model369 = pickle.load(open('../02_train/save_model/random_forest_classifier_369.sav','rb'))
model357 = pickle.load(open('../02_train/save_model/random_forest_classifier_357.sav','rb'))
model159 = pickle.load(open('../02_train/save_model/random_forest_classifier_159.sav','rb'))

In [5]:
test = [
    [1,3,5], # Win
    [2,4,6], # Lost
]

test = np.array(test)
test.shape
print(model456.predict(test))
print(model159.predict(test))

['win' 'lost']
['win' 'lost']


### Rede Neural - Model

##### Load <span style="color:red">Model</span>

In [6]:
model_rede = tf.keras.models.load_model('../02_train/save_model/model.h5')

##### Load <span style="color:red">WEIGHT</span>

In [7]:
model_rede.load_weights('../02_train/save_weight/weight.h5')

In [8]:
test = [
    [1,3,5,0,0,0.,2,4,0], # Win
    [1,3,0,5,0,0,2,4,6], # Lost
    [6,1,2,9,3,4,5,8,7], # Draw
    [2,4,0,0,0,0,1,3,5], # Win
    [1,3,0,0,5,0,2,4,6], # Lost
    [1,3,0,5,0,6,2,4,0], # Vou vencer na pŕoxima
    [0,0,0,0,0,0,0,1,0], # Indefinido (Perigoso dar Vitória.. isso não é bom)
    [2,0,3,0,0,0,0,1,0], # Indefinido (Perigoso dar Vitória.. isso não é bom)
    
]
test = pd.DataFrame(test)
test.shape

model_rede.predict(test)

array([[1.       ],
       [0.       ],
       [0.7825757],
       [1.       ],
       [0.       ],
       [0.       ],
       [1.       ],
       [1.       ]], dtype=float32)

In [9]:
# Vou criar a minha para treinar no erro das Forest (combinar)
model_rede_zero = tf.keras.models.Sequential()

model_rede_zero.add(tf.keras.layers.Dense(
    units = 10,
    activation = 'relu',
    input_shape = (9,)
))


model_rede_zero.add(tf.keras.layers.Dense(
    units = 10,
    activation = 'relu',
))


model_rede_zero.add(tf.keras.layers.Dense(
    units = 10,
    activation = 'relu',
))

model_rede_zero.add(tf.keras.layers.Dense(
    units = 1,
    activation = 'sigmoid',
))
model_rede_zero.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                100       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


### Funções

In [10]:
# função trnasformar String em Numero... (para treinar a rede)
def reverse_f(x):
    val = x[0]
    if val == 'win':
        return 1
    if val == 'lost':
        return 0
    return 0.5

In [11]:
# Função para transformar a saída do modelo Neural pra 'win','lost' ou'draw'
def f(x):
    val = x[0]
    #val = x
    if val >= 0.79:
        return 'win'
    if 0.40< val and val < 0.79:
        return 'draw'
    if val <= 0.40:
        return 'lost'

In [12]:
#pred_df['pred'] = pred_df[['pred']].apply(f,axis=1)

In [13]:
# creating the table
def df_f():
    col = []
    for n in range(1,10):
        col.append(n)
    col.append('resultado')

    df = pd.DataFrame(columns=col)
    return df

In [14]:
def n(x): # even odd function
    
    if x == 0:
        return 0.5 # para pular msm...
    
    elif x%2 == 0: # Número par retorna 0
        return x%2
    
    else:# número impar retorna 1
        return x%2

#even_odd(2)

In [15]:
# é utilizado para pegar valores de um vetor para os modelos por coluna

# def sortear_pos_model_col(self): Utilizado aqui

def get_col(vector, vector_3_col):
    new_vector = []
    for i in vector_3_col:
        new_vector.append(vector[i-1])
    return new_vector

"""x = [10,20,30,40,50,60,70,80,90]
x_model = [1,5,4]
# quero pegar o indice 1 , 2 e 9
x_new = get_col(x,x_model)
x_new""";

## GAME

In [16]:
class ticTacToe():
    
    def __init__(self,type_player='random'):
        
        self.df = df_f()
        self.pos ={
                    1:0,
                    2:0,
                    3:0,
                    4:0,
                    5:0,
                    6:0,
                    7:0,
                    8:0,
                    9:0,
                    'resultado':'unknown' }
        self.t = 1 # turnos
        
        self.model_treinar = model_rede_zero
        
        self.model = model_rede
        """self.model = self.criar_modelo() # cria o modelo"""
        
        
        self.type_player = type_player # default: 'random'
        
        
    
    #######################################################
    ################## RESTART GAME  ######################
    #######################################################
    def restart_df(self):
        self.df = df_f()
        
    
    def restart_game(self):
        self.pos ={
                    1:0,
                    2:0,
                    3:0,
                    4:0,
                    5:0,
                    6:0,
                    7:0,
                    8:0,
                    9:0,
                    'resultado':'unknown' }
        self.t = 1 # turnos
    #######################################################
        
        
    #######################################################
    ##################### TABLES ##########################
    #######################################################
    # Criar a Tabelas e dar appends nela
    def add_value_df(self):
        
        df_receber = pd.DataFrame( columns=list(self.pos.keys()) )
        for i in df_receber.columns:
            if i == 'resultado':
                df_receber[i] = [self.pos[i]]
            else:    
                df_receber[i] = [self.pos[int(i)]]
                
        self.df = self.df.append(df_receber,ignore_index=True)
    #########################################################

        
        
    #######################################################
    ##################### CHECKS ##########################
    #######################################################
    def check_defeat(self): # player é par
        if n(self.pos[1]) == n(self.pos[2]) == n(self.pos[3]) == 0:
            return 'break'
        if n(self.pos[4]) == n(self.pos[5]) == n(self.pos[6]) == 0:
            return 'break'
        if n(self.pos[1]) == n(self.pos[4]) == n(self.pos[7]) == 0:
            return 'break'
        if n(self.pos[1]) == n(self.pos[5]) == n(self.pos[9]) == 0:
            return 'break'
        if n(self.pos[2]) == n(self.pos[5]) == n(self.pos[8]) == 0:
            return 'break'
        if n(self.pos[3]) == n(self.pos[5]) == n(self.pos[7]) == 0:
            return 'break'
        if n(self.pos[3]) == n(self.pos[6]) == n(self.pos[9]) == 0:
            return 'break'
        if n(self.pos[7]) == n(self.pos[8]) == n(self.pos[9]) == 0:
            return 'break'
        return 'continue'

    def check_victory(self): # pc é impar
        if n(self.pos[1]) == n(self.pos[2]) == n(self.pos[3]) == 1:
            return 'break'
        if n(self.pos[4]) == n(self.pos[5]) == n(self.pos[6]) == 1:
            return 'break'
        if n(self.pos[1]) == n(self.pos[4]) == n(self.pos[7]) == 1:
            return 'break'
        if n(self.pos[1]) == n(self.pos[5]) == n(self.pos[9]) == 1:
            return 'break'
        if n(self.pos[2]) == n(self.pos[5]) == n(self.pos[8]) == 1:
            return 'break'
        if n(self.pos[3]) == n(self.pos[5]) == n(self.pos[7]) == 1:
            return 'break'
        if n(self.pos[3]) == n(self.pos[6]) == n(self.pos[9]) == 1:
            return 'break'
        if n(self.pos[7]) == n(self.pos[8]) == n(self.pos[9]) == 1:
            return 'break'
        return 'continue'
    
    def check_pos_full(self): # EMPATE
        if (self.pos[1] != 0 
            and self.pos[2] != 0 
            and self.pos[3] != 0 
            and self.pos[4] != 0
            and self.pos[5] != 0
            and self.pos[6] != 0
            and self.pos[7] != 0
            and self.pos[8] != 0
            and self.pos[9] != 0):
            
            return 'break'
    
        return 'continue'
    
    # Agora os turnos importam, pois turno impar é pc e turno par é player
    def check_turn(self):
        if self.t == 10: # Verificar tempo/turnos (f)
            return 'break'
        return 'continue'
    
    # Chamo esse método para verificar tudo
    def check(self):

        try:
            # Treina por turno
            self.treina_rede()
        except ValueError:
            pass
        
        # Verificar derrota
        if self.check_defeat() == 'break':
            #print('derrotado')
            self.pos['resultado'] = 'lost'
            
            return 'break'
        
        # Verificar Vitoria
        if self.check_victory() == 'break':
            #print('vencedor')
            self.pos['resultado'] = 'win'
            
            return 'break'
        
        # Verificar Turno
        self.t +=1
        
        if self.check_turn() == 'break':
            #print('turno')
            self.pos['resultado'] = 'draw' # Empate
            
            return 'break'
        
        # Verificar Tudo
        if self.check_pos_full() == 'break':
            #print('tudo')
            self.pos['resultado'] = 'draw' # Empate
            
            return 'break'
        
        return 'continue'
    #######################################################



    #######################################################
    ################ TIPO DE JOGADAS  #####################
    #######################################################
    # Aqui eu set os numeros no braço usando INPUTS
    def player(self, player):
        if player == 'player':
            move = int(input('player: '))
            if self.pos[move] == 0:
                self.pos[move] = int(self.t)
            else:
                while True:
                    move = int(input('player, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t)
                        break
                        
        if player == 'pc':
            move = int(input('pc: '))
            if self.pos[move] == 0:
                self.pos[move] = int(self.t)

            else:
                while True:
                    move = int(input('pc, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t)
                        break
    
    # Aqui eu uso para deixar para ser TREINADA
    def player_random(self, player):
        if player == 'player':
            move = self.sortear_pos_qualquer()
            if self.pos[move] == 0:
                self.pos[move] = int(self.t) # add o valor do turno 
            else:
                while True:
                    move = int(input('player, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t) # add o valor do turno
                        break
                        
        if player == 'pc':
            move = self.sortear_pos_qualquer()
            if self.pos[move] == 0:
                self.pos[move] = int(self.t) # add o valor do turno

            else:
                while True:
                    move = int(input('pc, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t) # add o valor do turno
                        break
                        

    # Modelo Aplicado
    def player_model(self,player):
        if player == 'player':
            
            # Fiz esse parâmetro no método construtor para não precisar ficar ajustando
            # no braço quando quiser testar algo randomico ou brincar com a máquina
            if self.type_player == 'input':
                move = int(input('player: ')) # Manual
            elif self.type_player == 'random':
                move = self.sortear_pos_qualquer() # Sortear
                
            
            if self.pos[move] == 0:
                self.pos[move] = int(self.t)
            else:
                while True:
                    move = int(input('player, tente novamente: '))

                    if self.pos[move] == 0:
                        self.pos[move] = int(self.t)
                        break
        
        
        if player == 'pc':
            
            move = self.modelos()
            #if self.pos[move] == 0:
            self.pos[move] = int(self.t)
            
    #######################################################
    
    
    
    
    
    #######################################################
    ################ SORTEIOS DE JOGADAS ##################
    #######################################################
    def sortear_pos_qualquer(self): # Ganhar tempo
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]

        return random.sample(livres,1)[0] # sortear 1 número dentro dos LIVRES

    def sortear_pos_boa(self):
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]

        
        # chuta um valor bom
        pos_ruins = [2,4,6,8] # Essas são posições ruins
        
        for ruin in pos_ruins:
            try:
                # Vamos remover as posições ruins dentro das Livres
                livres.remove(ruin)
                return random.sample(livres,1)[0]
            # não tem mais posição boa pra remover... então ele dá problema
            # joga a ruim msm...
            except ValueError:
                return random.sample(pos_ruins,1)[0]
                
    ########################################################
    
    ########################################################
    #################### ORDEM MODELOS #####################
    ########################################################
    def modelos(self):
        
        # Um modelo Chama o outro em caso de Fracasso
        
        # Modelo 1
        return self.sortear_pos_model_col(
            # Model 2
            next_model= self.sortear_pos_model_col(
                # Model 3
                next_model = self.sortear_pos_model_all(
                    next_model=self.sortear_pos_boa(),
                    function='draw' # Model 3
                ),
                function='draw' # Model 2
            ),
            function = 'draw' # Model 1
        )
    
    
    # Sequência boa No player RANDOM!!! Não funciona bem contra pessoas
    # modelo 1 - COL - function = win
    # modelo 2 - REDE - function = draw
    # modelo 3 - ALL - function = draw
    # sortear_pos_boa()
    
    # Sequência boa contra pessoas (buscar sempre o empate)
    # modelo 1 - COL - function = draw
    # modelo 2 - REDE - function = draw
    # modelo 3 - ALL - function = win
    # sortear_pos_boa()
    ##########################################################


    #######################################################
    ################ JOGADAS MODELO #######################
    #######################################################
    #def primeira_jogada(self):
            #return self.sortear_pos_boa()
            #return 5
            
    def sortear_pos_model_all(self,next_model,function): # Modelo 1 - Analisa todas colunas de uma vez
        
        
        
        # VALORES LIVRES
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]

        
        # pegar todos valoreS já jogados até o momento
        vetor = [list(self.pos.values())[:-1]] 

        # faz um loop com os valores livres
        # e coloca o valor em cada posição e roda o modelo
        
        for pos_livre in livres:
            vetor[0][pos_livre-1] = self.t

            
            pred = model_all.predict(vetor)
            #resultado = f(pred[0][0]) # Resultado (já transformado em 0 ;0.5 ou 1)
            
            
            resultado = pred[0]
            
            if resultado == function:# Se não for Derrota... já vale
                #print(resultado)
                return pos_livre
            
            vetor[0][pos_livre-1] = 0 #Voltar o valor vazio novamente para realizar o looping novamente
        
        
        # Se Esse modelo não der certo, vamos para o modelo 2 (col by col)
        return next_model
        
       
    def sortear_pos_model_col(self,next_model,function): # Modelo 2 - analisa só as colunas importantes
        
        # Primeira Jogada
        #if self.t == 1:
        #    return self.primeira_jogada()
        
        # VALORES LIVRES
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]

        
        # pegar todos valoreS já jogados até o momento
        vetor = [list(self.pos.values())[:-1]] 

        # Os modelos estão todos colocados em um vetor
        models = [model123,model456,model789,model147,model258,model369,model357,model159]
        
        # para chamá-los um por um e testar
        for model in models:
            if model is model123:
                vetor_model =[1,2,3]
            if model is model456:
                vetor_model =[4,5,6]
            if model is model789:
                vetor_model =[7,8,9]
            if model is model147:
                vetor_model =[1,4,7]
            if model is model258:
                vetor_model =[2,5,8]
            if model is model369:
                vetor_model =[3,6,9]
            if model is model357:
                vetor_model =[3,5,7]
            if model is model159:
                vetor_model =[1,5,9]
                
        
            for pos_livre in livres:
                vetor[0][pos_livre-1] = self.t
                
                vetor_new = get_col(vetor[0],vetor_model)
                
                # pega somente 3 colunas
                pred = model.predict([vetor_new])
                resultado = pred[0]
                
                
                if resultado == function:# Se não for Derrota... já vale
                    
                    return pos_livre
                
                vetor[0][pos_livre-1] = 0 #Voltar o valor vazio novamente para realizar o looping novamente
        
        
        # Se nada der certo vamos sortear um 
        return next_model
    
    
    def sortear_pos_model_rede(self,next_model,function):
        
        #print('estou sendo usada :D')
        
        # VALORES LIVRES
        livres =[n for n in list(self.pos.keys())[:-1] if list(self.pos.values())[n-1]== 0]


        # pegar todos valoreS já jogados até o momento
        vetor = [list(self.pos.values())[:-1]] 


        for pos_livre in livres:
            vetor[0][pos_livre-1] = self.t


            pred = self.model.predict(vetor)

            #resultado = f(pred[0][0]) # Resultado (já transformado em 0 ;0.5 ou 1)
            resultado = f(pred[0]) # Preciso usar a função para separar em WIN,LOST,DRAW
            
            #print(resultado)

            if resultado == function:# Se não for Derrota... já vale
                
                return pos_livre

            vetor[0][pos_livre-1] = 0 


        # Se nada der certo vamos sortear um valor bom
        # pelo menos
        return next_model

    
    ########################################################
    
    
    ########################################################
    ############## Treinamentos REDE NEURAL ################
    ########################################################
    # Irei treinar só nos erros das redes forest all and col jogados contra mim
    """def criar_modelo(self):
        
        # Se não existir uma Rede Neural pronta na pasta save_model / seve_weight
        #.
        #.
        #.
        
        
        # Então... Criar uma Rede Neural
        model = tf.keras.models.Sequential()
        
        model.add(tf.keras.layers.Dense(
            units = 10,
            activation = 'relu',
            input_shape = (9,)
        ))


        model.add(tf.keras.layers.Dense(
            units = 10,
            activation = 'relu',
        ))

        #model.add(tf.keras.layers.Dropout(0.1))

        model.add(tf.keras.layers.Dense(
            units = 10,
            activation = 'relu',
        ))

        model.add(tf.keras.layers.Dense(
            units = 1,
            activation = 'sigmoid',
        ))
        model.summary()
        
        return model"""
    
    def treina_rede(self):
        # ESSE MÉTODO TREINA DURANTE A PARTIDA!!!! turno por turno ele é chamado


        #self.model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
        self.model_treinar.compile(loss ='huber_loss' ,optimizer = 'RMSprop' , metrics = ['accuracy'])
        
        
        # Dados 
        x = np.asarray(self.df.drop('resultado',axis=1)).astype(np.float32)
        y = np.asarray(self.df[['resultado']].apply(reverse_f,axis=1) ).astype(np.float32)
        
        # Treinar
        history = self.model_treinar.fit(
        x,
        y,
        validation_split=None,#CUIDADO QUANDO TIVER POUCOS DADOS
        epochs =1,
        verbose = 1,# Não printar
        )
        
        history.history.keys()
    #############################################################
    
    
    
        
    # RUN DE PROGRAM
    def run(self):
        
        while True:
            
            # Jogar Player
            #self.player('pc')
            #self.player_random('pc')
            self.player_model('pc')
             
            # Check -----------------
            if self.check() == 'break':
                break
            
            
            # Jogar PC
            #self.player('player')
            #self.player_random('player')
            self.player_model('player')
             
            # Check -----------------
            if self.check() == 'break':
                break
            #------------------------
        
        
        
            
        self.add_value_df() # Add valores
        
        #if self.df.shape[0] > 10:
        #    self.treina_rede()
        #if self.pos['resultado'] == 'lost':
        #    self.treina_rede()
        #self.treina_rede()
        


In [17]:
#game = ticTacToe(type_player = 'random')
#game = ticTacToe(type_player = 'player')
game = ticTacToe()

In [27]:
for i in range(0,5):
    game.type_player='random' # Para nao precisar resetar tudo toda tabela
    game.restart_game()
    game.run()
game.df.tail()


1/1 [==============================] - 1s 613ms/step - loss: 0.0609 - accuracy: 0.5000


1/1 [==============================] - 1s 604ms/step - loss: 0.0334 - accuracy: 0.5000


1/1 [==============================] - 1s 601ms/step - loss: 0.0683 - accuracy: 0.5000


1/1 [==============================] - 1s 591ms/step - loss: 0.0394 - accuracy: 0.4286


1/1 [==============================] - 1s 594ms/step - loss: 0.0211 - accuracy: 0.6250


...

2/2 [==============================] - 1s 4ms/step - loss: 0.0227 - accuracy: 0.7077


2/2 [==============================] - 1s 4ms/step - loss: 0.0179 - accuracy: 0.7014


2/2 [==============================] - 1s 3ms/step - loss: 0.0199 - accuracy: 0.7271


2/2 [==============================] - 1s 3ms/step - loss: 0.0201 - accuracy: 0.6763


,1,2,3,4,5,6,7,8,9,resultado
47,7,0,5,9,1,6,2,3,4,draw
48,1,0,5,0,3,0,6,4,2,lost
49,0,0,0,5,3,1,0,4,2,win
50,9,4,6,2,0,5,8,7,1,draw
51,2,0,1,0,0,5,4,0,3,win


In [19]:
print('win: ',game.df[game.df['resultado']=='win'].count()['resultado'],' draw: ',game.df[game.df['resultado']=='draw'].count()['resultado'],' lost: ',game.df[game.df['resultado']=='lost'].count()['resultado'])

win:  0  draw:  0  lost:  1


In [28]:
## Jogar no input
game.type_player = 'input'
game.restart_game()
game.run()
game.df.tail()

2/2 [==============================] - 1s 6ms/step - loss: 0.0218 - accuracy: 0.7115


KeyboardInterrupt: Interrupted by user

In [21]:
#print('draw',game.df[game.df['resultado']=='draw'].count()['resultado'])

In [22]:
#print('lost',game.df[game.df['resultado']=='lost'].count()['resultado'])

In [23]:
# Save TABLE
#game.df.to_pickle('./dados.pkl')

In [24]:
#load_df = pd.read_pickle('./dados.pkl')

In [25]:
#load_df.drop_duplicates().reset_index().drop('index',axis=1)

In [ ]:
## Remover Valores duplicados
#dub = game.df.drop_duplicates().reset_index().drop('index',axis=1)
#dub.tail()